In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [121]:
outcomes = pd.read_excel("sample_data/WinningPartyByCounties.xlsx") # party outcome at county level

rawdemo2012 = pd.read_excel("sample_data/demographicsallstates2012.xlsx") # demographic variables at county level 2012
rawdemo2016 = pd.read_excel("sample_data/demographicsallstates2016.xlsx") # demographic variables at county level 2016
rawdemo2020 = pd.read_excel("sample_data/demographicsallstates2020.xlsx") # demographic variables at county level 2020

wipoverty = pd.read_csv("sample_data/wipoverty.csv") # poverty variable at county level -- Wisconsin
wipoverty = wipoverty.drop([0,1, 38, 39]) # Drop national and state level statistics

prediction = pd.read_excel("sample_data/2022datafinal.xlsx")
prediction.head(72)

,GIS Join Match Code,Data File Year,State/US-Abbreviation (USPS),Region Code,Division Code,State Name,State Code,County Name,County Code,County Subdivision Code,...,Aggregate income in the past 12 months (in 2022 inflation-adjusted dollars),Total.5,In labor force,In labor force Percentage,In labor force: Civilian labor force,In labor force: Civilian labor force: Employed,In labor force: Civilian labor force: Unemployed,In labor force: Armed Forces,Not in labor force,Not in labor force Percentage
0,G0400010,2018-2022,AZ,NaN,NaN,Arizona,4,Apache County,1,NaN,...,2076142800,47146,28106,0.596148,27163,26411,752,943,19040,0.403852
1,G0400030,2018-2022,AZ,NaN,NaN,Arizona,4,Cochise County,3,NaN,...,9081671900,189569,111766,0.589580,111079,107254,3825,687,77803,0.410420
2,G0400050,2018-2022,AZ,NaN,NaN,Arizona,4,Coconino County,5,NaN,...,581565400,20399,8968,0.439629,8968,8452,516,0,11431,0.560371
3,G0400070,2018-2022,AZ,NaN,NaN,Arizona,4,Gila County,7,NaN,...,555512400,18046,9028,0.500277,9028,8242,786,0,9018,0.499723
4,G0400090,2018-2022,AZ,NaN,NaN,Arizona,4,Graham County,9,NaN,...,1716337800,47261,26459,0.559849,26434,24856,1578,25,20802,0.440151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,G1301070,2018-2022,GA,NaN,NaN,Georgia,13,Emanuel County,107,NaN,...,143656900,3061,2415,0.788958,2411,2305,106,4,646,0.211042
68,G1301090,2018-2022,GA,NaN,NaN,Georgia,13,Evans County,109,NaN,...,247062000,4404,3701,0.840372,3669,3559,110,32,703,0.159628
69,G1301110,2018-2022,GA,NaN,NaN,Georgia,13,Fannin County,111,NaN,...,13531938900,228369,159676,0.699202,150082,142335,7747,9594,68693,0.300798
70,G1301130,2018-2022,GA,NaN,NaN,Georgia,13,Fayette County,113,NaN,...,479373300,12669,7862,0.620570,7827,6562,1265,35,4807,0.379430


In [122]:
# Filter files for Wisconsin

wioutcomes = outcomes[outcomes['state'] == 'WISCONSIN'] # filtered outcome variable for Wisconsin

demo2012 = rawdemo2012[rawdemo2012['State Name'] == 'Wisconsin']
demo2016 = rawdemo2016[rawdemo2016['State Name'] == 'Wisconsin']
demo2020 = rawdemo2020[rawdemo2020['State Name'] == 'Wisconsin'] # filtered demographic variables for Wisconsin

In [123]:
# Separate outcomes & poverty by year

wioutcomes2012 = wioutcomes[wioutcomes['year'] == 2012]
wioutcomes2016 = wioutcomes[wioutcomes['year'] == 2016]
wioutcomes2020 = wioutcomes[wioutcomes['year'] == 2020] # separated outcomes variable by year

wipoverty2012 = wipoverty[wipoverty['Year'] == 2012]
wipoverty2016 = wipoverty[wipoverty['Year'] == 2016]
wipoverty2020 = wipoverty[wipoverty['Year'] == 2020] # separated poverty variable by year

In [124]:
# Merge

election2012 = pd.merge(demo2012, wipoverty2012, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2012 = pd.merge(election2012, wioutcomes2012, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2012
# now have a complete file for 2012

election2016 = pd.merge(demo2016, wipoverty2016, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2016 = pd.merge(election2016, wioutcomes2016, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2016
# now have a complete file for 2016

election2020 = pd.merge(demo2020, wipoverty2020, left_on='County Name', right_on='Name', how='outer') # Merge poverty with demographics
election2020 = pd.merge(election2020, wioutcomes2020, left_on='ID', right_on='county_fips', how='outer') # Merge outcomes with election2012
# now have a complete file for 2020

In [125]:
# Prepare prediction data

nvprediction = prediction[prediction['State Name'] == 'Nevada']

nvprediction = nvprediction[['County Name','Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage',]]

nvprediction.head(100)
# Prediction data ready. Missing poverty statistics and median income statistics

,County Name,Males Under 18 Percentage,Males 18-24 Percentage,Males 25-34 Percentage,Males 35-44 Percentage,Males 45-64 Percentage,Males 65+ Percentage,Females Under 18 Percentage,Females 18-24 Percentage,Females 25-34 Percentage,...,Bachelor's degree Percentage,Master's degree Percentage,Professional school degree Percentage,Doctorate degree Percentage,"less than $39,999 Percentage","$40,000-$74,999 Percentage","$75,000-$99,999 Percentage","$100,000-$149,999 Percentage","$150,000-$199,999 Percentage","more than $200,000 Percentage"
257,Churchill County,0.115471,0.048408,0.072337,0.058680,0.129403,0.091739,0.590696,0.040025,0.058719,...,0.123830,0.029402,0.019868,0.003311,0.552288,0.235294,0.094118,0.060131,0.011765,0.046405
258,Clark County,0.116119,0.043159,0.074449,0.071219,0.126333,0.070759,0.609010,0.041047,0.072395,...,0.174020,0.062964,0.017342,0.009622,0.493536,0.291255,0.085932,0.074525,0.033460,0.021293
259,Douglas County,0.080787,0.031308,0.049074,0.053359,0.142574,0.149466,0.567790,0.028458,0.043132,...,0.204155,0.076050,0.021445,0.011662,0.271894,0.313618,0.132967,0.179734,0.030261,0.071527
260,Elko County,0.136549,0.047425,0.077724,0.073937,0.126511,0.061549,0.603694,0.041660,0.069925,...,0.105023,0.046283,0.013207,0.005565,0.349017,0.278398,0.136753,0.143684,0.036451,0.055696
261,Esmeralda County,0.039796,0.011224,0.032653,0.046939,0.252041,0.157143,0.518367,0.000000,0.044898,...,0.152348,0.090493,0.000000,0.000000,0.224505,0.214371,0.125553,0.175416,0.107207,0.152949
262,Eureka County,0.126387,0.038224,0.033909,0.048089,0.128853,0.128853,0.629470,0.016646,0.028360,...,0.097210,0.054905,0.000000,0.000000,0.353586,0.200844,0.071730,0.222785,0.108861,0.042194
263,Humboldt County,0.133152,0.040716,0.073439,0.063362,0.129561,0.081548,0.608885,0.038747,0.059307,...,0.154423,0.041538,0.006703,0.005468,0.092028,0.128725,0.112047,0.217593,0.168977,0.280630
264,Lander County,0.131983,0.049581,0.071404,0.081355,0.139665,0.068785,0.588163,0.014665,0.051152,...,0.066165,0.033212,0.033991,0.007006,0.180392,0.208773,0.120345,0.185770,0.130153,0.174567
265,Lincoln County,0.120036,0.042157,0.062791,0.086532,0.165298,0.081429,0.542268,0.030175,0.053694,...,0.091622,0.061814,0.000000,0.000000,0.121715,0.143473,0.117301,0.226193,0.176477,0.214841
266,Lyon County,0.108976,0.033600,0.068932,0.065601,0.135493,0.105645,0.582199,0.032590,0.059140,...,0.115436,0.034886,0.005621,0.005180,0.211024,0.240763,0.141567,0.194593,0.101704,0.110348


In [126]:
elections = pd.concat([election2012, election2016, election2020], axis=0, ignore_index=True)
finalelections = elections[['party', 'Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage']]

In [127]:
# Train-test split
from sklearn.model_selection import train_test_split

# Process data:
X = finalelections[['Males Under 18 Percentage','Males 18-24 Percentage','Males 25-34 Percentage',
                           'Males 35-44 Percentage', 'Males 45-64 Percentage', 'Males 65+ Percentage', 'Females Under 18 Percentage',
                           'Females 18-24 Percentage', 'Females 25-34 Percentage', 'Females 35-44 Percentage', 'Females 45-64 Percentage',
                           'Females 65+ Percentage',
                           'White alone Percentage', 'Black or African American alone Percentage', 'American Indian and Alaska Native alone Percentage',
                           'Asian alone Percentage', 'Native Hawaiian and Other Pacific Islander alone Percentage',
                           'Some other race alone Percentage', 'Two or more races Percentage', 'Two or more races: Two races including Some other race Percentage',
                           'Two or more races: Two races excluding Some other race, and three or more races Percentage',
                           'No schooling completed Percentage', 'Nursery school Percentage','Kindergarten Percentage', '1st grade Percentage', '2nd grade Percentage',
                           '3rd grade Percentage', '4th grade Percentage', '5th grade Percentage', '6th grade Percentage', '7th grade Percentage', '8th grade Percentage',
                           '9th grade Percentage', '10th grade Percentage', '11th grade Percentage', '12th grade, no diploma Percentage', 'Regular high school diploma Percentage',
                           'GED or alternative credential Percentage', 'Some college, less than 1 year Percentage', 'Some college, 1 or more years, no degree Percentage',
                           "Associate's degree Percentage", "Bachelor's degree Percentage", "Master's degree Percentage", 'Professional school degree Percentage',
                           'Doctorate degree Percentage',
                           'less than $39,999 Percentage', '$40,000-$74,999 Percentage', '$75,000-$99,999 Percentage', '$100,000-$149,999 Percentage', '$150,000-$199,999 Percentage', 'more than $200,000 Percentage']]
y = finalelections['party']

# Train-test split:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2,random_state=100)
print(f"X_train shape: {X_train.shape}") # got it done

X_train shape: (177, 51)


In [128]:
finalelections.head(50)


,party,Males Under 18 Percentage,Males 18-24 Percentage,Males 25-34 Percentage,Males 35-44 Percentage,Males 45-64 Percentage,Males 65+ Percentage,Females Under 18 Percentage,Females 18-24 Percentage,Females 25-34 Percentage,...,Bachelor's degree Percentage,Master's degree Percentage,Professional school degree Percentage,Doctorate degree Percentage,"less than $39,999 Percentage","$40,000-$74,999 Percentage","$75,000-$99,999 Percentage","$100,000-$149,999 Percentage","$150,000-$199,999 Percentage","more than $200,000 Percentage"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DEMOCRAT,0.155131,0.059666,0.126934,0.129143,0.311058,0.218068,0.174544,0.054460,0.076682,...,0.077429,0.031156,0.005469,0.004056,0.461124,0.356578,0.091721,0.065155,0.017978,0.007443
3,DEMOCRAT,0.239632,0.102185,0.110256,0.110628,0.290787,0.146511,0.220995,0.093064,0.108450,...,0.162342,0.050513,0.010527,0.008034,0.513374,0.300558,0.105085,0.059083,0.012934,0.008965
4,REPUBLICAN,0.232498,0.080220,0.114787,0.118628,0.291899,0.161968,0.210710,0.068306,0.109839,...,0.108270,0.038102,0.011019,0.002693,0.455602,0.324987,0.112572,0.074855,0.017254,0.014729
5,DEMOCRAT,0.191652,0.059576,0.083226,0.108038,0.355389,0.202119,0.183379,0.046458,0.081608,...,0.169215,0.069800,0.020879,0.007775,0.451739,0.299380,0.123936,0.086712,0.021930,0.016304
6,REPUBLICAN,0.257153,0.097893,0.140714,0.137927,0.266301,0.100013,0.240872,0.098651,0.133810,...,0.185207,0.054809,0.012198,0.006207,0.368734,0.304051,0.140255,0.121074,0.034725,0.031160
7,DEMOCRAT,0.219172,0.072912,0.105447,0.120842,0.313580,0.168046,0.223157,0.066977,0.097312,...,0.124362,0.032809,0.008541,0.008228,0.417455,0.337014,0.119523,0.094988,0.017701,0.013319
8,REPUBLICAN,0.203529,0.053056,0.084436,0.105356,0.326906,0.226717,0.184512,0.053075,0.083630,...,0.121663,0.039956,0.008128,0.004962,0.506453,0.304442,0.101073,0.058824,0.014808,0.014400
9,REPUBLICAN,0.273257,0.075837,0.120005,0.144313,0.282236,0.104352,0.260150,0.059581,0.122504,...,0.188705,0.058441,0.014579,0.002855,0.270409,0.311989,0.182452,0.169755,0.037929,0.027466


In [129]:
finalelections = finalelections.drop([0,1])

In [130]:
finalelections.head(50)


,party,Males Under 18 Percentage,Males 18-24 Percentage,Males 25-34 Percentage,Males 35-44 Percentage,Males 45-64 Percentage,Males 65+ Percentage,Females Under 18 Percentage,Females 18-24 Percentage,Females 25-34 Percentage,...,Bachelor's degree Percentage,Master's degree Percentage,Professional school degree Percentage,Doctorate degree Percentage,"less than $39,999 Percentage","$40,000-$74,999 Percentage","$75,000-$99,999 Percentage","$100,000-$149,999 Percentage","$150,000-$199,999 Percentage","more than $200,000 Percentage"
2,DEMOCRAT,0.155131,0.059666,0.126934,0.129143,0.311058,0.218068,0.174544,0.054460,0.076682,...,0.077429,0.031156,0.005469,0.004056,0.461124,0.356578,0.091721,0.065155,0.017978,0.007443
3,DEMOCRAT,0.239632,0.102185,0.110256,0.110628,0.290787,0.146511,0.220995,0.093064,0.108450,...,0.162342,0.050513,0.010527,0.008034,0.513374,0.300558,0.105085,0.059083,0.012934,0.008965
4,REPUBLICAN,0.232498,0.080220,0.114787,0.118628,0.291899,0.161968,0.210710,0.068306,0.109839,...,0.108270,0.038102,0.011019,0.002693,0.455602,0.324987,0.112572,0.074855,0.017254,0.014729
5,DEMOCRAT,0.191652,0.059576,0.083226,0.108038,0.355389,0.202119,0.183379,0.046458,0.081608,...,0.169215,0.069800,0.020879,0.007775,0.451739,0.299380,0.123936,0.086712,0.021930,0.016304
6,REPUBLICAN,0.257153,0.097893,0.140714,0.137927,0.266301,0.100013,0.240872,0.098651,0.133810,...,0.185207,0.054809,0.012198,0.006207,0.368734,0.304051,0.140255,0.121074,0.034725,0.031160
7,DEMOCRAT,0.219172,0.072912,0.105447,0.120842,0.313580,0.168046,0.223157,0.066977,0.097312,...,0.124362,0.032809,0.008541,0.008228,0.417455,0.337014,0.119523,0.094988,0.017701,0.013319
8,REPUBLICAN,0.203529,0.053056,0.084436,0.105356,0.326906,0.226717,0.184512,0.053075,0.083630,...,0.121663,0.039956,0.008128,0.004962,0.506453,0.304442,0.101073,0.058824,0.014808,0.014400
9,REPUBLICAN,0.273257,0.075837,0.120005,0.144313,0.282236,0.104352,0.260150,0.059581,0.122504,...,0.188705,0.058441,0.014579,0.002855,0.270409,0.311989,0.182452,0.169755,0.037929,0.027466
10,REPUBLICAN,0.234566,0.078282,0.134184,0.135922,0.290126,0.126921,0.233306,0.072417,0.120252,...,0.131980,0.036733,0.009172,0.004481,0.388566,0.336062,0.130300,0.099648,0.029751,0.015674
11,REPUBLICAN,0.291945,0.082150,0.111168,0.115786,0.260589,0.138362,0.289355,0.070303,0.105279,...,0.081154,0.024693,0.007349,0.002465,0.460063,0.337079,0.104792,0.065046,0.018727,0.014293


In [131]:
# Linear interpolation for all numerical columns (it basically creates data points depending on surrounding data points in order to fill NaN's since the value is not a true 0 or NaN)
finalelections = finalelections.interpolate(method='linear', axis=0)

# Check if there are still any NaNs remaining
remaining_nans = finalelections.isna().sum().sum()
print(f"Remaining NaNs after interpolation: {remaining_nans}")


Remaining NaNs after interpolation: 4


<ipython-input-131-d2fc40ee11b8>:2: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  finalelections = finalelections.interpolate(method='linear', axis=0)


In [137]:
# Drop rows with any NaN values
finalelections_cleaned_rows = finalelections.dropna(axis=0)

# Check the shapes after dropping rows or columns
print("Shape after dropping rows with NaNs:", finalelections_cleaned_rows.shape)

Shape after dropping rows with NaNs: (220, 52)


In [133]:
# To check to see if there are any NaN's
print(finalelections.isna().sum().sum())  # Should be 0

4


In [134]:
# Ensure all columns are properly inferred
finalelections = finalelections.infer_objects()

# Interpolate numeric columns
finalelections = finalelections.interpolate(method='linear', axis=0)

# Fill any remaining NaN values with 0
finalelections = finalelections.fillna(0)

# Check if any NaNs remain
remaining_nans = finalelections.isna().sum().sum()
print(f"Remaining NaNs after filling with 0: {remaining_nans}")

Remaining NaNs after filling with 0: 0


<ipython-input-134-232c7de7f9c9>:5: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  finalelections = finalelections.interpolate(method='linear', axis=0)


In [135]:
# Building the model
from sklearn.ensemble import RandomForestClassifier  # Import RandomForestClassifier
import pandas as pd

# Impute missing values in X_train with the mean of each column
X_train = X_train.fillna(X_train.mean())

# Impute missing values in y_train with the most frequent value
y_train = y_train.fillna(y_train.mode()[0])

# Verify if any NaN values still exist
print(f"NaNs in X_train: {X_train.isna().sum().sum()}")
print(f"NaNs in y_train: {y_train.isna().sum().sum()}")

NaNs in X_train: 0
NaNs in y_train: 0


In [138]:
# Building the model
from sklearn.ensemble import RandomForestClassifier

# Fit model:
model = RandomForestClassifier() # Build a random forest model
rf = model.fit(X_train,y_train) # Fit the rf model